In [76]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_excel("swap14.xlsx")
data = data[data['request_id'] != 'uint64']
data.head()

,group_id,in_test,inactivity_time,next_query_distance,next_query_time,query,request_id,user_id,verdict,verdict_fixed
1,1,True,NaN,0.572011,154918,красивые сборки тарков,1,7122,Не решил,Не решил
2,1,True,NaN,0,254,русское порно с разговорами,1,15325,Решил вкладкой,Решил вкладкой
3,1,True,NaN,0.44038896241944037,151708,русское порно с разговорами,2,15325,Решил -1,Решил вкладкой
4,1,False,395,0.2873228583998181,398,porno365,1,406,Решил 0,Решил 0
5,2,False,NaN,NaN,NaN,русское порно,2,406,Решил 0,Решил 0


In [3]:
data.groupby('in_test')['user_id'].nunique()

in_test
False    2562
True     2553
Name: user_id, dtype: int64

In [5]:
data[data['verdict_fixed'] == 'Решил -1']

,group_id,in_test,inactivity_time,next_query_distance,next_query_time,query,request_id,user_id,verdict,verdict_fixed


In [6]:
data.groupby('in_test')['verdict_fixed'].value_counts()

in_test  verdict_fixed         
False    Решил 0                   3362
         Не решил                  2770
         Решил 1                   1778
         Технический               1035
         Решил бескликово           868
         Решил 2                    793
         Решил вкладкой             506
         Решил 3                    437
         Решил 4                    294
         Решил 5                    218
         Решил 6                    132
         Решил 7                     94
         Решил 11+                   94
         Решил 8                     82
         Решил 9                     68
         Решил в правой колонке      62
         Решил 10                    45
True     Решил 0                   2956
         Не решил                  2939
         Решил 1                   1644
         Технический                999
         Решил 2                    949
         Решил бескликово           907
         Решил 3                    682
        

In [52]:
users = list(set(data['user_id']))
np.random.seed(26)
buckets = np.random.randint(0, 100, len(users))
data['bucket'] = data['user_id'].apply(lambda x: buckets[users.index(x)])

In [53]:
verdict_to_score = {'Не решил': -1,
                    'Решил бескликово': 1.5,
                    'Решил вкладкой': 1,
                    'Решил в правой колонке': 1,
                    'Технический': 0,
                    'Решил 0': 1,
                    'Решил 1': 1/2,
                    'Решил 2': 1/3,
                    'Решил 3': 1/4,
                    'Решил 4': 1/5,
                    'Решил 5': 1/6,
                    'Решил 6': 1/8,
                    'Решил 7': 1/8,
                    'Решил 8': 1/9,
                    'Решил 9': 1/10,
                    'Решил 10': 1/11,
                    'Решил 11+': 1/12}
data['score'] = data['verdict_fixed'].apply(lambda x: verdict_to_score[x])

In [54]:
control = data[data['in_test'] == False].groupby(['bucket'])['score'].sum()
test = data[data['in_test']].groupby(['bucket'])['score'].sum()

In [55]:
from scipy.stats import mannwhitneyu

In [56]:
mannwhitneyu(control, test)

MannwhitneyuResult(statistic=6073.5, pvalue=0.008747771251673352)

In [57]:
sum(control), sum(test)

(3875.8020202020225, 3494.7702020202023)

In [ ]:
control = data[data['in_test'] == False].groupby(['bucket'])[''].sum()
test = data[data['in_test']].groupby(['bucket'])['score'].sum()

In [60]:
data['is_solved'] = data['verdict_fixed'].apply(lambda x: x not in ['Не решил', 'Технический'])
data['is_not_solved'] = data['verdict_fixed'].apply(lambda x: x == 'Не решил')

In [61]:
control = data[data['in_test'] == False].groupby(['bucket'])['is_solved'].sum()
test = data[data['in_test']].groupby(['bucket'])['is_solved'].sum()

In [62]:
sum(control), sum(test)

(8833, 8932)

In [63]:
mannwhitneyu(control, test)

MannwhitneyuResult(statistic=4974.5, pvalue=0.9512845468932765)

In [64]:
control = data[data['in_test'] == False].groupby(['bucket'])['is_not_solved'].sum()
test = data[data['in_test']].groupby(['bucket'])['is_not_solved'].sum()

In [65]:
sum(control), sum(test)

(2770, 2939)

In [66]:
mannwhitneyu(control, test)

MannwhitneyuResult(statistic=4661.5, pvalue=0.4086833630972362)

In [ ]:
control = data[data['in_test'] == False].groupby(['bucket'])['is_not_solved'].sum()
test = data[data['in_test']].groupby(['bucket'])['is_not_solved'].sum()

In [73]:
tc = data[data['in_test'] == False].groupby(['bucket'])[['is_not_solved', 'is_solved']].sum()
control = tc['is_solved'] / (tc['is_not_solved'] + tc['is_solved'])
tt = data[data['in_test']].groupby(['bucket'])[['is_not_solved', 'is_solved']].sum()
test = tt['is_solved'] / (tt['is_not_solved'] + tt['is_solved'])

In [74]:
sum(tc['is_solved']) / sum((tc['is_not_solved'] + tc['is_solved'])), sum(tt['is_solved']) / sum((tt['is_not_solved'] + tt['is_solved']))

(0.7612686374213565, 0.7524218684188358)

In [75]:
mannwhitneyu(control, test)

MannwhitneyuResult(statistic=5553.5, pvalue=0.17662857263661402)